In [6]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [7]:
class AgentState(TypedDict):
    Number_1: int
    Number_2: int
    Number_3: int
    Number_4: int
    Result_1: str
    Result_2: str
    operation_1: str
    operation_2: str

In [8]:
def addition_1(state: AgentState) -> AgentState:
    result = state["Number_1"] + state["Number_2"]
    state["Result_1"] = str(result)
    return state

def subtraction_1(state: AgentState) -> AgentState:
    result = state["Number_1"] - state["Number_2"]
    state["Result_1"] = str(result)
    return state

def addition_2(state: AgentState) -> AgentState:
    result = int(state["Result_1"]) + state["Number_3"] + state["Number_4"]
    state["Result_2"] = str(result)
    return state

def subtraction_2(state: AgentState) -> AgentState:
    result = int(state["Result_1"]) - state["Number_3"] - state["Number_4"]
    state["Result_2"] = str(result)
    return state

def Decide_Next_Node_1(state: AgentState) -> str:
    if state['operation_1'] == "+":
        return "Add_1"
    elif state['operation_1'] == "-":
        return "Sub_1"
    else:
        raise ValueError("Unsupported Operation! Use '+' or '-'.")

def Decide_Next_Node_2(state: AgentState) -> str:
    if state['operation_2'] == "+":
        return "Add_2"
    elif state['operation_2'] == "-":
        return "Sub_2"
    else:
        raise ValueError("Unsupported Operation! Use '+' or '-'.")

def final_node(state: AgentState) -> AgentState:
    print("Final result:", state["Result_2"])
    return state

In [13]:
builder = StateGraph(AgentState)

# Add nodes
builder.add_node("add_1", addition_1)
builder.add_node("sub_1", subtraction_1)
builder.add_node("add_2", addition_2)
builder.add_node("sub_2", subtraction_2)
builder.add_node("final", final_node)
builder.add_node("Decider_1", Decide_Next_Node_1)
builder.add_node("Decider_2", Decide_Next_Node_2)


builder.add_conditional_edges(
    START,
    Decide_Next_Node_1,
    {
        "Add_1": "add_1",
        "Sub_1": "sub_1"
    }
)

# Add edges from add_1 and sub_1 to Decider_2
builder.add_edge("add_1", "Decider_2")
builder.add_edge("sub_1", "Decider_2")

# Add conditional edges from Decider_2 to add_2 or sub_2
builder.add_conditional_edges(
    "Decider_2",
    Decide_Next_Node_2,
    {
        "Add_2": "add_2",
        "Sub_2": "sub_2"
    }
)

# Add edges from add_2 and sub_2 to final
builder.add_edge("add_2", "final")
builder.add_edge("sub_2", "final")

# Compile the graph
graph = builder.compile()